In [1]:
# coding: utf-8

In [2]:
from __future__ import print_function

import os
import sys
import time
from datetime import timedelta

import numpy as np
import tensorflow as tf
from sklearn import metrics

from dataOwn import *
import pickle
from CNNDiseaseModel import *

In [3]:
try:
    bool(type(unicode))
except NameError:
    unicode = str

In [4]:
class CnnModel:
    def __init__(self):
        
        self.config = TCNNConfig()
        self.vocabulary_word2index, self.vocabulary_index2word  = create_voabulary(self.config.word2vec_model_path)
        self.config.vocab_size = len(self.vocabulary_word2index)+1
        #这里通过实际的word2vec模型统计词典中词的数量，赋值到config中，然后加载RNN模型
        self.model = CNNDisease(self.config)
        print(self.config.vocab_size)
    
        save_dir = 'checkpoints/textrnn'
        save_path = os.path.join(save_dir, 'best_validation')  # 最佳验证结果保存路径

    
        self.session = tf.Session()
        self.session.run(tf.global_variables_initializer())
        saver = tf.train.Saver()
        saver.restore(sess=self.session, save_path=save_path)  # 读取保存的模型

    def predict(self, message):
        # 支持不论在python2还是python3下训练的模型都可以在2或者3的环境下运行
        #content = unicode(message)
        x_temp = list([message.strip().split(" ") ])
        x = [[a.strip() for a in b]  for b in x_temp]
        for i in range(len(x)):
            for j in range(len(x[i])):
                x[i][j] = self.vocabulary_word2index.get(x[i][j],0)
        data = np.array(x).tolist()
        
        #data = [self.vocabulary_word2index[x] for x in content if x in self.vocabulary_word2index]

        feed_dict = {
            self.model.input_x: pad_sequences(data, self.config.sequence_length),
            self.model.dropout_keep_prob: 1.0
        }

        y_pred_cls = self.session.run(self.model.y_pred_cls, feed_dict=feed_dict)
        return y_pred_cls[0]
    def feed_data(self,x_batch, y_batch, keep_prob):
        feed_dict = {
            self.model.input_x: x_batch,
            self.model.input_y: y_batch,
            self.model.dropout_keep_prob: keep_prob
        }
        return feed_dict
    def evaluate(self, x, y):
        """评估在某一数据上的准确率和损失"""
        #batch_eval = batch_iter(x_, y_, config.batch_size)
        batch_size = self.config.batch_size
        total_loss = 0.0
        total_acc = 0.0
        eval_out = []
        data_len = len(x)
        num_batch = int((data_len - 1) / batch_size) + 1

        indices = np.random.permutation(np.arange(data_len))
        print("data_len=", data_len, " num_batch=", num_batch)
        x = np.array(x)
        y = np.array(y)
        x_shuffle = x[indices]
        y_shuffle = y[indices]

        for i in range(num_batch):
            start_id = i * batch_size
            end_id = min((i + 1) * batch_size, data_len)
            x_batch, y_batch =  x_shuffle[start_id:end_id], y_shuffle[start_id:end_id]

            batch_len = len(x_batch)
            feed_dict = self.feed_data(x_batch, y_batch, 1.0)
            loss, acc, y_pred_cls = self.session.run([self.model.loss, self.model.acc, self.model.y_pred_cls], feed_dict=feed_dict)
            total_loss += loss * batch_len
            total_acc += acc * batch_len
            eval_out = np.concatenate([eval_out, y_pred_cls])

        #这里为了获取shuffle之后的y，直接把得到批的过程拿过来了
        return total_loss / data_len, total_acc / data_len, eval_out, y_shuffle
    def test(self, x_val, y_val):

        start_time = time.time()
        print('Testing...')
        total_loss, total_acc, eval_out, y_real = self.evaluate( x_val, y_val)
        print("total_loss=", total_loss, " total_acc=", total_acc)
        print(metrics.classification_report(y_real,eval_out))

        cm = metrics.confusion_matrix(y_real, eval_out)
        print("cm====\n", cm)  
    def predictBatch(self, test_data_path, test_label_path):
        x_val, y_val = loadTrainOrTest_data_oneLabel(test_data_path, test_label_path, self.vocabulary_word2index)
        x_val = pad_sequences(x_val, self.config.sequence_length)  # padding to max length   \
        self.test(x_val, y_val)
        

In [5]:
cnn_model = CnnModel()

232015
INFO:tensorflow:Restoring parameters from checkpoints/textrnn/best_validation


In [6]:
dicPath = "../datasets/firstCode2Index2TypeNew.txt"
typeDict = dict()
for item in open(dicPath,"r").readlines():
    itemArr = item.split(" ")
    typeDict[int(itemArr[1])] = itemArr[2]

In [8]:
test_demo = ['胃窦 凹陷性浮肿 粘膜隆起 慢性浅表性胃炎 HP 反跳痛 无压痛 电子胃镜 胃底 超声内镜检查 脾未触及 十二指肠球炎 病理性杂音 黄斑瘤 肺呼吸音 湿性啰音 巩膜无黄染 查体 入院 心率  胆汁 神志 下肢 糜烂 中年 腹部 平坦 辅助 女性 患者 检查 精神 建议',
             '腹部 疼痛 阴性 患者 反酸 血管杂音 震水音 移动性浊音 里急后重 蠕动波 胃肠型 未触及 腹部叩诊音 反跳痛 右下腹压痛 右下腹部 肝脾肋下未触及 无明显诱因 腹壁静脉曲张 转移性右 下腹痛 鼓音 周围部 肠鸣音 上腹 阵发性 入院 腰痛 腹胀 胸闷 心悸 心慌 大便 放射 腹泻 未见 头痛 恶心 持续性 呕吐 发热 平坦 加重 转移 固定 紧张',
            '未触及 转移性右下腹痛 血常规 腹部包块 腹部B超 肠型 移动性浊音 肋下 入院查体 反跳痛 右下腹压痛 肠鸣音 心肺听诊无异常 腹肌 囊肿 入院 神志 身体健康 未见 腹部 平坦 辅助 患者 检查 小时 精神',
            '叩痛 肋下 反跳痛 肝脾 深压痛 右下腹痛 未触及 胃肠型 检查日期 蠕动波 肾区 检查单位 移动性浊音 肠鸣音 右下腹 查体 腹肌 入院 未及 未见 腹部 平坦'
            ,'入院查体 轻度黄染 肌张力 双肺呼吸音清 肠鸣音 湿性啰音 小时 心音 入院 患儿 外貌 主因 肤色 新生儿 腹泻 哭声 发热 四肢 腹部 增强'
            ,'急诊 黄染 无畸形 瓣膜听诊区 腹平软 干湿罗音 心肌酶 血钾 握持反射 指(趾)端 足月新生儿 觅食反射 前囟 肺呼吸音 吸吮反射 皮肤干燥 肝脾肋下未触及 拥抱反射 肌张力'
            ]
#输入文本
'''
test_data_path = "../cnnDatasets/testAdmin.feature"
test_label_path = "../cnnDatasets/testAdmin.label"
x_val, y_val = loadTrainOrTest_data_oneLabel_Source(test_data_path, test_label_path, cnn_model.vocabulary_word2index)
count = 0
for index in range(0,1500):
    item = x_val[index]
    if(y_val[index] == cnn_model.predict(item)):
        count +=1
    print(y_val[index],cnn_model.predict(item), typeDict[cnn_model.predict(item)])
print("count=", count)
'''
for i in test_demo:
    print(cnn_model.predict(i), typeDict[cnn_model.predict(i)])
#输入测试文件地址
test_data_path = "../cnnDatasets/testAdmin.feature"
test_label_path = "../cnnDatasets/testAdmin.label"
#cnn_model.predictBatch(test_data_path, test_label_path)

7 肿瘤

7 肿瘤

8 泌尿生殖系统疾病

7 肿瘤

8 泌尿生殖系统疾病

7 肿瘤

